### Ejercicio 1


In [ ]:
import pandas as pd
# Cargar los datos
df = pd.read_csv('train_data.csv')
muestra_por_clase = 2000 #Toma una muestra x datos por cada clase para el entrenamiento
df_muestra = df.groupby('clase', group_keys=False).apply(lambda x: x.sample(min(len(x), muestra_por_clase)))

df_muestra.reset_index(drop=True, inplace=True) # Se guarda el dataset de la muestra tomada
df_muestra.to_csv('muestra1.csv', index=False) 


In [17]:
# Carga del modelo y definición de la función de preprocesamiento
import spacy
nlp = spacy.load('en_core_web_sm') 

def preprocess_text_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)


In [18]:
# Preprocesamiento, se guardan los tokens resultantes en una columna para no estar preprocesado cada vez
df = pd.read_csv('muestra1.csv')
df['tokens'] = df['text'].apply(preprocess_text_spacy)
df.to_csv('muestra1_tokens.csv', index=False)

In [19]:
# Eliminar filas con valores NaN en la columna 'tokens'
df = pd.read_csv('muestra1_tokens.csv')
df = df.dropna(subset=['tokens'])


0
Index(['date', 'id_news', 'media_outlet', 'title', 'text', 'url', 'year',
       'clase', 'tokens'],
      dtype='object')


In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Vectorización
vectorizer = CountVectorizer(max_features=5000)
X_bow = vectorizer.fit_transform(df['tokens']).toarray()

In [21]:
from sklearn.model_selection import train_test_split
# Division del dataset para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_bow, df['clase'], test_size=0.2, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Como contaba con las etiquetes usé un enfoque supervisado
# Como los datos textuales suelen tener una alta dimensionalidad, ya que cada palabra se convierte en una característica.
# random Forest maneja bien la alta dimensionalidad porque cada árbol trabaja solo con un subconjunto aleatorio 
# de las características.

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

print(f"Accuracy Random Forest: {accuracy_score(y_test, y_pred_rf)}")
print(classification_report(y_test, y_pred_rf))

Accuracy Random Forest: 0.6158578263841422
                 precision    recall  f1-score   support

     accidentes       0.67      0.75      0.71       344
        ciencia       0.66      0.46      0.54       436
       deportes       0.56      0.54      0.55       400
       economia       0.57      0.66      0.61       415
      educacion       0.58      0.53      0.56       406
entretenimiento       0.55      0.53      0.54       397
  internacional       0.69      0.85      0.76       396
 medio_ambiente       0.57      0.62      0.59       411
       politica       0.68      0.67      0.68       403
          salud       0.60      0.42      0.50       401
     tecnologia       0.64      0.79      0.71       380

       accuracy                           0.62      4389
      macro avg       0.62      0.62      0.61      4389
   weighted avg       0.61      0.62      0.61      4389



In [23]:
import joblib
# Guardar el modelo en un archivo local para poder reutilizar
joblib.dump(model_rf, 'Model_Ej1_final.pkl')
joblib.dump(vectorizer, 'Vect_Ej1_final.pkl')


['Vect_Ej1_final.pkl']

## Probar el modelo

In [7]:
import joblib
import pandas as pd
import spacy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
nlp = spacy.load('en_core_web_sm') 

def preprocess_text_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

# Carga el modelo y el vectorizador
modelo = joblib.load('Model_Ej1_final.pkl') #Entrenamiento con 20k de datos -> 40min
vectorizador = joblib.load('Vect_Ej1_final.pkl')

# Carga el dataset
df = pd.read_csv('train_data.csv') # remplazar datos para probar

# En caso de tener datos no etiquedatos con "clase" comentar esto ->
muestra_por_clase = 5  # Ajustar este número según sea necesario
df_news = df.groupby('clase', group_keys=False).apply(
    lambda x: x.sample(min(len(x), muestra_por_clase), random_state=70).sample(frac=1, random_state=70)  
)

# hasta aqui y descomentar lo siguiente
#df_news = df.sample(n=10, random_state=70)

# Preprocesa los textos
df_news["tokens"] = df_news['text'].apply(preprocess_text_spacy)

# Transforma los textos con el vectorizador
X_transformed = vectorizador.transform(df_news["tokens"])

# Realiza la predicción con el modelo
df_news["val_predict"] = modelo.predict(X_transformed)

#print(df_news.head())





C:\Users\leonardo\AppData\Local\Temp\ipykernel_8468\2762299510.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_news = df.groupby('clase', group_keys=False).apply(


In [9]:
# Verificar resultados manualmente
# Comentar si no hay datos etiquetados
y_true = df_news["clase"]  # Etiquetas verdaderas
y_pred = df_news["val_predict"]  # Predicciones hechas por el modelo

# Imprimir el classification report
print(classification_report(y_true, y_pred))

filas_iguales = (df_news['clase'] == df_news['val_predict']).sum()
total_filas = df_news.shape[0]
print(f"\n Predicciones acertadas:  {filas_iguales} de {total_filas}, aprox un {(filas_iguales/total_filas):.2f}% \n")
print(df_news[['clase', 'val_predict']])
df_news.reset_index(drop=True, inplace=True)
df_news.to_csv('ej1_resultados.csv', index=False)



                 precision    recall  f1-score   support

     accidentes       0.83      1.00      0.91         5
        ciencia       1.00      1.00      1.00         5
       deportes       0.75      0.60      0.67         5
       economia       0.60      0.60      0.60         5
      educacion       1.00      0.80      0.89         5
entretenimiento       0.80      0.80      0.80         5
  internacional       1.00      1.00      1.00         5
 medio_ambiente       1.00      1.00      1.00         5
       politica       1.00      1.00      1.00         5
          salud       0.83      1.00      0.91         5
     tecnologia       1.00      1.00      1.00         5

       accuracy                           0.89        55
      macro avg       0.89      0.89      0.89        55
   weighted avg       0.89      0.89      0.89        55


 Predicciones acertadas:  49 de 55, aprox un 0.89% 

              clase      val_predict
0        accidentes       accidentes
1        accid

In [6]:
print("")